In [ ]:
!pip install scikeras

In [ ]:
# Importing necesaries libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import normalize

from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
# Loading the MNIST dataset, which contains handwritten digits
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
# Display the first image in the training set (optional)
X_train[0]

In [ ]:
# Normalizing the training and test datasets to have values between 0 and 1
X_train = normalize(X_train, axis=1)
X_test = normalize(X_test, axis=1)

In [ ]:
# Building the neural network model
model = Sequential([
    # Input layer with the shape of the images
    Input(shape=(28, 28)),
    # Flattening the 2D input into a 1D vector
    Flatten(),
    # First hidden layer with 32 neurons and ReLU activation function
    Dense(32, activation='relu'),
    # Output layer with 10 neurons (one for each class) and softmax activation
    Dense(10, activation='softmax'),
])

# Compiling the model with Adam optimizer,
# sparse categorical cross-entropy loss function, and accuracy as the metric
model.compile(
    optimizer=Adam(learning_rate=0.005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
# Training the model on the training data for 15 epochs
model.fit(X_train, y_train, epochs= 15)

In [ ]:
# Evaluating the model on the test dataset
model.evaluate(X_test, y_test)

# Hyperparameter Tuning

In [ ]:
# Function to build a customizable neural network model
def build_model(n_hidden=1, n_neurons=32, learning_rate=0.001, kernel_initializer='he_normal' ,add_dropout=False, add_batch_normalization=False):
    # Initialize the sequential model
    model = Sequential()
    # Add the input layer with the shape of the images
    model.add(Input(shape=(28, 28)))
    # Flattening the 2D input into a 1D vector
    model.add(Flatten())

    # Add hidden layers based on the number specified in n_hidden
    for layer in range(n_hidden):
        # Add Dense layer with n_neurons and specified kernel initializer
        model.add(Dense(n_neurons, activation='relu', kernel_initializer=kernel_initializer))
        if add_dropout:
            model.add(Dropout(0.2))
        if add_batch_normalization:
            model.add(BatchNormalization())
    # Add the output layer with 10 neurons and softmax activation
    model.add(Dense(10, activation='softmax'))

    # Compile the model using Adam optimizer with the specified learning rate,
    # loss function, and metrics
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
# Building a model with 3 hidden layers and 64 neurons in each hidden layer
build_model(n_hidden=3, n_neurons=64)

In [ ]:
# Wrapping the Keras model with KerasClassifier for compatibility with scikit-learn
keras_clf = KerasClassifier(build_model)

In [ ]:
# Training the model on the training data for 15 epochs
keras_clf.fit(X_train, y_train, epochs=15)

In [ ]:
# Evaluating the model on the test data
keras_clf.score(X_test, y_test)

In [ ]:
# Defining the hyperparameter grid for grid search
param_grid = {
    'model__n_hidden': [3, 4],
    'model__n_neurons': [128, 256],
    #'model__learning_rate': [0.001, 0.005, 0.01],
    #'model__kernel_initializer': ['he_normal', 'glorot_uniform'],
    #'model__use_dropout': [True, False],
    #'model__use_batch_normalization': [True, False]
}

# Wrapping the model with KerasClassifier again for use with GridSearchCV
keras_clf = KerasClassifier(build_model)

# Performing grid search with 3-fold cross-validation and verbosity set to 2
# (for detailed output)
grid_search = GridSearchCV(keras_clf, param_grid, cv=3, verbose=2)

In [ ]:
# Running the grid search on the training data
grid_search.fit(X_train, y_train)

In [ ]:
# Getting the best model from the grid search
grid_search.best_estimator_

In [ ]:
# Assigning the best model found to the variable 'model'
model = grid_search.best_estimator_

In [ ]:
# Getting the best hyperparameters found during the grid search
grid_search.best_params_

In [ ]:
# Evaluating the best model on the test data
model.score(X_test, y_test)